In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
!ls /gdrive

'My Drive'


In [ ]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!apt install swig
!pip install stable-baselines[mpi]==2.10.0
!pip install git+https://github.com/rynsy/gym-sokoban

TensorFlow 1.x selected.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 33 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 2s (690 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.

In [ ]:
import gym
import gym_sokoban
import numpy as np
 
from stable_baselines.deepq.policies import LnCnnPolicy 
from stable_baselines.common.vec_env import DummyVecEnv 
from stable_baselines import DQN
from stable_baselines.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold, CheckpointCallback, CallbackList
 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
"""
  Need to save or render environments as you train to see learning rate per level
"""
path = F"/gdrive/My Drive/DQN/" 
STEPS_PER_SAVE = 10_000
 
env = gym.make('Sokoban-small-v0')
model = DQN(LnCnnPolicy, env, 
        tensorboard_log=path+"GRAPH/", 
        double_q=True,
        prioritized_replay=True,
        prioritized_replay_alpha=0.99,
        learning_starts=1000,
        verbose=1
        )







Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where









In [ ]:
for i in range(1000):
  print("=" * 50, " ENVIRONMENT #", i, " ", "=" * 50)
  env.reset(regenerate=True)
  model.set_env(env)
  callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=-9, verbose=1)
  eval_callback = EvalCallback(env, callback_on_new_best=callback_on_best, verbose=1)
  callback_checkpoint = CheckpointCallback(save_freq=STEPS_PER_SAVE, save_path=path+"MODEL/", name_prefix="dqn_pr", verbose=1)
  callback_list = CallbackList([eval_callback, callback_checkpoint])
  model.learn(total_timesteps=STEPS_PER_SAVE, callback=callback_list)
  model.save(path+"MODEL/final_model_for_env_"+str(i))

==================================================  ENVIRONMENT # 0   ==================================================


/usr/local/lib/python3.6/dist-packages/stable_baselines/common/callbacks.py:277: UserWarning: Training and eval env are not of the same type<SokobanEnv_Small0<Sokoban-small-v0>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7f6cf1334630>
  "{} != {}".format(self.training_env, self.eval_env))


Eval num_timesteps=10000, episode_reward=-20.00 +/- 0.00
Episode length: 200.00 +/- 0.00
New best mean reward!
==================================================  ENVIRONMENT # 1   ==================================================


/usr/local/lib/python3.6/dist-packages/stable_baselines/common/callbacks.py:277: UserWarning: Training and eval env are not of the same type<SokobanEnv_Small0<Sokoban-small-v0>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7f6cf13347f0>
  "{} != {}".format(self.training_env, self.eval_env))


KeyboardInterrupt: ignored

In [12]:
def test_model(env, regen=False):
  test_rewards = []
  for i in range(10):
    test_reward = []
    obs = env.reset(regenerate=regen)
    while True:
      action, _states = model.predict(obs)
      obs, rewards, done, info = env.step(action)
      test_reward.append(rewards)
      if done:
        break
    test_reward = np.sum(test_reward)
    test_rewards.append(test_reward)
    print("Test {}: {}".format(i,test_reward))
  if regen:
    print("Average reward for random-board tests: {}".format(np.average(test_rewards)))
  else:
    print("Average reward for same-board tests: {}".format(np.average(test_rewards)))
  return np.average(test_rewards)

In [ ]:
STEPS = 10_000
env = gym.make('Sokoban-small-v0')
model = DQN(LnCnnPolicy, env, 
     #   tensorboard_log=path+"GRAPH/", 
        double_q=True,
        prioritized_replay=True,
        prioritized_replay_alpha=0.99,
        learning_starts=int(STEPS*0.5),
        verbose=2
        )
board_learned = True
for i in range(1000):
  if board_learned:
    print("=" * 50, " ENVIRONMENT #", i, " ", "=" * 50)
  env.reset(regenerate=board_learned)
  model.set_env(env)
  model.learn(total_timesteps=STEPS)
  reward = test_model(env)
  board_learned = reward > 9
  if board_learned:
    test_model(env, regen=board_learned)
    model.save(path+"MODEL/final_model_for_env_"+str(i))
    print("=" * 50, " ENVIRONMENT #", i, " HAS BEEN LEARNED ", "=" * 50)


==================================================  ENVIRONMENT # 0   ==================================================
Test 0: -19.0
Test 1: -19.0
Test 2: -19.0
Test 3: -19.0
Test 4: -19.0
Test 5: -19.0
Test 6: -19.0
Test 7: -19.0
Test 8: -19.0
Test 9: -19.0
Average reward for same-board tests: -19.0
